In [1]:
#libraries
import cv2
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
import time

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


can I start from specific timestamps ?

what happens for skip_frames=0 ?

In [2]:
#read video file frame by frame
def read_video(video,skip_frames,resolution_width):
    resolution_height=round(int(resolution_width * 9/16))
    resolution=(resolution_width,resolution_height)
    vid = cv2.VideoCapture(video)
    frames=[]
    vid_length=0
    while(vid.isOpened()):
        # Capture frame-by-frame
        ret, frame = vid.read() # if ret is false, frame has no content
        if not ret:
            break
        if vid_length%skip_frames==0: # skip every "skip_frame"
            frame=cv2.resize(frame,resolution) # resize the video to a different resolution
            frames.append(frame) #add the individual frames to a list
        vid_length+=1 #increase the vid_length counter
    vid.release()
    cv2.destroyAllWindows()
    return frames

Dominant Color Extractor

- add better comments

In [3]:
def extract_dominant_color(frame_list,bin_threshold=0.05,colors_to_return=5):
    print(str(len(frame_list))+' frames to process.')
    start=time.time()
    rgb_to_color=fn_rgb_to_color() #get the color dict 
    bins={} #bins dict for histograms 
    for rgb in rgb_to_color: #init the dict with zeros for every key
        bins[rgb_to_color[rgb]]=0
    rgb_list=[] #create a traverseable list of the rgb_values
    for rgb in rgb_to_color: #map the values of the dict to a list
        rgb_list.append(rgb)
    i = 0
    for image in frame_list: #traverse the video
        #flatten the image to 1d 
        img = image.reshape((image.shape[0] * image.shape[1], 3))     
        for pixel in img: # do nearest neighbour search on every pixel every color in the list
            bin_aux=[]
            #get the euclidean distance between the colors and the current pixel
            for rgb in rgb_list:
                bin_aux.append(euclidean(pixel,rgb))
            # get the index of the color,which has the smallest distance, in rgb_list
            min_pos = np.argmin(bin_aux)
            #increment the respective color 
            bins[rgb_to_color[rgb_list[min_pos]]]+=1
        i+=1
        end=time.time()
        print('Finished '+str(i)+',time: '+str(end-start))
    #create a dataframe, sorted descending by count
    bins_sorted=sorted(zip(list(bins.values()),list(bins.keys())),reverse=True)
    df=pd.DataFrame(bins_sorted,columns=['count','color'])
    df.set_index('color',inplace=True) #set the colors as the index of the dataframe
    norm_factor = len(frame_list)* np.shape(frame_list[0])[0] * np.shape(frame_list[0])[1]  #normalize the bins
    df=df/norm_factor 
    bin_threshold=bin_threshold/100
    df = df[df>bin_threshold].dropna() #kick bins from the dataframe with precentage lower than bin_threshold 
    return df.head(colors_to_return)#return the color_return highest bins, default 5, if less bins then
                                #color_return are there return all

In [4]:
def fn_rgb_to_color():
    colors={'darkred':(139,0,0),
    'firebrick':(178,34,34),
    'crimson':(220,20,60),
    'red':(255,0,0),
    'tomato':(255,99,71),
    'salmon':(250,128,114),
    'darkorange':(255,140,0),
    'gold':(255,215,0),
    'darkkhaki':(189,183,107),
    'yellow':(255,255,0),
    'darkolivegreen':(85,107,47),
    'olivedrab':(107,142,35),
    'greenyellow':(173,255,47),
    'darkgreen':(0,100,0),
    'aquamarine':(127,255,212),
    'steelblue':(70,130,180),
    'skyblue':(135,206,235),
    'darkblue':(0,0,139),
    'blue':(0,0,255),
    'royalblue':(65,105,225),
    'purple':(128,0,128),
    'violet':(238,130,238),
    'deeppink':(255,20,147),
    'pink':(255,192,203),
    'antiquewhite':(250,235,215),
    'saddlebrown':(139,69,19),
    'sandybrown':(244,164,96),
    'ivory':(255,255,240),
    'dimgrey':(105,105,105),
    'grey':(28,128,128),
    'silver':(192,192,192),
    'lightgrey':(211,211,211),
    'black':(0,0,0),
    'white':(255,255,255),
    'darkcyan':(0,139,139),
    'cyan':(0,255,255),
    'green':(0,128,0),
    'khaki':(240,230,140),
    'goldenrod':(218,165,32),
    'orange':(255,165,0),
    'coral':(255,127,80),
    'magenta':(255,0,255),
    'wheat':(245,222,179),
    'skin':(255,224,189),
    'purple4':(147,112,219)}
    rgb_to_color={}
    for color in colors:
        rgb_to_color[colors[color]]=color
    #purple4 is median purple
    #skin is caucasian
    return rgb_to_color

In [5]:
#read .json-file
import json
with open('/home/jacob/Downloads/ImCo___Colour_Range.json') as file:
    ground_truth=json.load(file)
ground_truth['annotations'][0]['content']

'white, black, sandybrown, lightgrey, dimgrey, antiquewhite'

In [6]:
#embed the color strings as unique numbers 
color_embedding = {}
rgb_to_color = fn_rgb_to_color()
i=0
for rgb in rgb_to_color:
    color_embedding[rgb_to_color[rgb]]=i
    i+=1

In [7]:
#use the embedding on the ground truth
g_t_all_embedded = []
for g_t in ground_truth['annotations']:
    g_t=g_t['content']
    g_t_embedded=[]
    for color in g_t.replace(" ","").split(','):
        g_t_embedded.append(color_embedding[color])
    g_t_all_embedded.append(g_t_embedded)
# g_t_all_embedded

In [8]:
#precision & recall
from sklearn.metrics import recall_score,precision_score
recall_score(g_t_embedded,g_t_embedded,average='micro')
precision_score(g_t_embedded,g_t_embedded,average='micro')

1.0

In [9]:
(ground_truth['annotations'][0]['end']-1080940)/1000*25

493.0

In [10]:
gt_e_s_list =[]
for i,content in enumerate(ground_truth['annotations']):
    gt_e_s=[]
    gt_e_s.append(round(content['begin']/1000*25))
    gt_e_s.append(round(content['end']/1000*25))
    gt_e_s_list.append(gt_e_s)

read random segments of the video

In [11]:
#read video file frame by frame
def read_video_segments(video,start_frame,end_frame,resolution_width):
    resolution_height=round(int(resolution_width * 9/16))
    resolution=(resolution_width,resolution_height)
    vid = cv2.VideoCapture(video)
    frames=[]
    vid_length=0
    while(vid.isOpened()):
        # Capture frame-by-frame
        ret, frame = vid.read() # if ret is false, frame has no content
        if not ret:
            break
        # skip every "skip_frame"
        if vid_length>=start_frame:
            # resize the video to a different resolution
            frame=cv2.resize(frame,resolution)
            frames.append(frame) #add the individual frames to a list
        if vid_length==end_frame:
            break
        vid_length+=1 #increase the vid_length counter
    vid.release()
    cv2.destroyAllWindows()
    return frames

In [13]:
segment_list=[]
segment_list.append(read_video_segments('/home/jacob/Downloads/Wells_John_CompanyMen_full.mp4',
                    gt_e_s_list[0][0],
                    gt_e_s_list[0][1],90))
segment_list.append(read_video_segments('/home/jacob/Downloads/Wells_John_CompanyMen_full.mp4',
                    gt_e_s_list[34][0],
                    gt_e_s_list[34][1],90))
segment_list.append(read_video_segments('/home/jacob/Downloads/Wells_John_CompanyMen_full.mp4',
                    gt_e_s_list[50][0],
                    gt_e_s_list[50][1],90))
# for gt_i in gt_e_s_list:
#     segment_list.append(read_video_segments('/home/jacob/Downloads/Wells_John_CompanyMen_full.mp4',
#                    gt_i[0],gt_i[1],200))

In [14]:
dominant_colors_list=[]
dominant_colors_list.append(extract_dominant_color(segment_list[0]))
# for segment in segment_list:
#     dominant_colors_list.append(extract_dominant_color(segment))

494 frames to process.
Finished 1,time: 2.3474111557006836
Finished 2,time: 4.6415696144104
Finished 3,time: 6.938725471496582
Finished 4,time: 9.260047197341919
Finished 5,time: 11.670360803604126
Finished 6,time: 14.04411005973816
Finished 7,time: 16.598532915115356
Finished 8,time: 18.948460817337036
Finished 9,time: 21.441291570663452
Finished 10,time: 23.79943060874939
Finished 11,time: 26.12184190750122
Finished 12,time: 28.867852926254272
Finished 13,time: 31.51576328277588
Finished 14,time: 34.11748003959656
Finished 15,time: 36.78745698928833
Finished 16,time: 39.36446666717529
Finished 17,time: 41.81240892410278
Finished 18,time: 44.265801668167114
Finished 19,time: 46.698641777038574
Finished 20,time: 49.13616490364075
Finished 21,time: 51.57817983627319
Finished 22,time: 53.97883152961731
Finished 23,time: 56.40056395530701
Finished 24,time: 58.80626845359802
Finished 25,time: 61.24051833152771
Finished 26,time: 63.69020986557007
Finished 27,time: 66.2995548248291
Finished 

Finished 222,time: 541.6476356983185
Finished 223,time: 544.0622217655182
Finished 224,time: 546.475697517395
Finished 225,time: 548.9011492729187
Finished 226,time: 551.312249660492
Finished 227,time: 553.7197999954224
Finished 228,time: 556.1343421936035
Finished 229,time: 558.5487303733826
Finished 230,time: 560.9915397167206
Finished 231,time: 563.4129283428192
Finished 232,time: 565.8365592956543
Finished 233,time: 568.2862911224365
Finished 234,time: 570.7130234241486
Finished 235,time: 573.2267408370972
Finished 236,time: 575.8015089035034
Finished 237,time: 578.3863418102264
Finished 238,time: 580.8871743679047
Finished 239,time: 583.3040194511414
Finished 240,time: 585.7739210128784
Finished 241,time: 588.1728193759918
Finished 242,time: 590.6050064563751
Finished 243,time: 593.0489213466644
Finished 244,time: 595.434159040451
Finished 245,time: 597.929895401001
Finished 246,time: 600.5028712749481
Finished 247,time: 603.1223633289337
Finished 248,time: 605.7237539291382
Finis

Finished 444,time: 1080.3846685886383
Finished 445,time: 1082.9449763298035
Finished 446,time: 1085.3613946437836
Finished 447,time: 1087.8090362548828
Finished 448,time: 1090.2445404529572
Finished 449,time: 1092.6597185134888
Finished 450,time: 1095.0738775730133
Finished 451,time: 1097.458402633667
Finished 452,time: 1099.8724451065063
Finished 453,time: 1102.2958452701569
Finished 454,time: 1104.719627380371
Finished 455,time: 1107.1355800628662
Finished 456,time: 1109.54359126091
Finished 457,time: 1111.966955423355
Finished 458,time: 1114.3846700191498
Finished 459,time: 1116.8307917118073
Finished 460,time: 1119.2532413005829
Finished 461,time: 1121.6442646980286
Finished 462,time: 1124.0735557079315
Finished 463,time: 1126.52104306221
Finished 464,time: 1128.9336290359497
Finished 465,time: 1131.3077776432037
Finished 466,time: 1133.7682793140411
Finished 467,time: 1136.3244335651398
Finished 468,time: 1138.865693807602
Finished 469,time: 1141.2207584381104
Finished 470,time: 1

In [15]:
dominant_colors_list.append(extract_dominant_color(segment_list[1]))
dominant_colors_list.append(extract_dominant_color(segment_list[2]))

29 frames to process.
Finished 1,time: 2.558013677597046
Finished 2,time: 5.10842752456665
Finished 3,time: 7.6607208251953125
Finished 4,time: 10.201727867126465
Finished 5,time: 12.755277395248413
Finished 6,time: 15.313477754592896
Finished 7,time: 17.86403751373291
Finished 8,time: 20.40081000328064
Finished 9,time: 22.959739685058594
Finished 10,time: 25.530656099319458
Finished 11,time: 28.17780351638794
Finished 12,time: 30.821110248565674
Finished 13,time: 33.581899642944336
Finished 14,time: 36.3972544670105
Finished 15,time: 39.252577781677246
Finished 16,time: 42.01441144943237
Finished 17,time: 44.72531747817993
Finished 18,time: 47.494871377944946
Finished 19,time: 50.01935625076294
Finished 20,time: 52.542256116867065
Finished 21,time: 55.14281725883484
Finished 22,time: 57.71587014198303
Finished 23,time: 60.25458312034607
Finished 24,time: 62.87325382232666
Finished 25,time: 65.51970791816711
Finished 26,time: 68.29300427436829
Finished 27,time: 70.97787284851074
Finish

In [16]:
pred_list=[]
for dominant_colors in dominant_colors_list:
    embedding=[]
    for color in dominant_colors.index.tolist():
        embedding.append(color_embedding[color])
    pred_list.append(embedding)
# pred_list

In [ ]:
# precision_score(g_t_all_embedded[6],pred_list[6],average='micro')

In [23]:
# g_t_all_embedded
print(ground_truth['annotations'][0]['content'])
print(ground_truth['annotations'][34]['content'])
print(ground_truth['annotations'][50]['content'])

white, black, sandybrown, lightgrey, dimgrey, antiquewhite
black, skin, ivory, lightgrey, gold
white, black, skin, grey


In [26]:
# pred_list
print(dominant_colors_list[0])
print(dominant_colors_list[1])
print(dominant_colors_list[2])

                   count
color                   
black           0.528510
dimgrey         0.182825
darkolivegreen  0.113512
silver          0.066971
lightgrey       0.061281
                   count
color                   
dimgrey         0.505456
black           0.294628
silver          0.081778
darkkhaki       0.043180
darkolivegreen  0.042253
              count
color              
silver     0.397647
black      0.262880
dimgrey    0.134625
lightgrey  0.132405
darkkhaki  0.026559


In [ ]:
color_embedding